## Save ensemble mean netcdf of ACCESS-ESM1.5 and ACCESS-CM2
- data can be made available for users in a sandbox for more comparisons
- same for CMIP6 multimodel mean from an ESMValTool recipe

In [1]:
## set path location to save data
outpath = '/scratch/iq82/fc6164/esmvaltool_output/process_data/'

In [2]:
from esmvalcore.dataset import Dataset
import iris

# a template for dataset
var = Dataset(
    short_name='ts', mip='Amon', project='CMIP6',
    activity='CMIP',  dataset='ACCESS-CM2',
    ensemble='*',
    institute='*', grid='*',
    exp='historical', timerange='*'
)

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.10/lib/python3.11/site-packages/distributed/diagnostics/nvml.py:14: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml


We can use the `add_supplementary` so that `esmvalcore.dataset` can handle finding the data for these [supplementary variables](https://docs.esmvaltool.org/projects/ESMValCore/en/latest/recipe/preprocessor.html#supplementary-variables-ancillary-variables-and-cell-measures).

We have to load a single ensemble first to load these as `cell_measure` and `ancillary_variable` of the cube which we can then add to the ensemble cube. Loading supplementaries for an ensemble `Dataset` will try to concatenate the supplementary variables

In [3]:
## add supplementary cell measures for single ensemble 
supp_var = var.copy()
supp_var.add_supplementary(short_name='areacella', mip='fx')
supp_var.add_supplementary(short_name='sftlf', mip="fx")

firstcb = list(supp_var.from_files())[0].load()
area = firstcb.cell_measure('cell_area')
land = firstcb.ancillary_variable('land_area_fraction')

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.10/lib/python3.11/site-packages/esmvalcore/config/_config.py:59: ESMValCoreDeprecationWarning: Usage of extra facets located in ~/.esmvaltool/extra_facets has been deprecated in ESMValCore version 2.13.0 and is scheduled for removal in version 2.15.0. Please use the configuration option `extra_facets` instead (see https://github.com/ESMValGroup/ESMValCore/pull/2747 for details). To silent this warning and ignore deprecated extra facets, set the environment variable ESMVALTOOL_USE_NEW_EXTRA_FACETS_CONFIG=1.
  warn_if_old_extra_facets_exist()
(for file /g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r10i1p1f1/fx/sftlf/gn/v20220819/sftlf_fx_ACCESS-CM2_historical_r10i1p1f1_gn.nc)


In [4]:
# load ensemble mean cubes
ens_var_cubes = {}
for shortname in ['ts','pr']:
    dataset = var.copy(short_name=shortname)
    ens_var_cubes[shortname] = dataset.load()

In [5]:
## add supplementaries to loaded ensemble cube

for short_name, cube in ens_var_cubes.items():
    cube.add_cell_measure(area, firstcb.cell_measure_dims(area))
    cube.add_ancillary_variable(land, firstcb.ancillary_variable_dims(land))
    iris.save(cube, outpath + f'{short_name}_ACCESS-CM2_ensemble.nc')

In [6]:
## test loading back with xarray or iris
import xarray as xr
# iris.load( outpath + f'{short_name}_ACCESS-CM2_ensemble.nc')
xr.open_dataset(outpath + f'{short_name}_ACCESS-CM2_ensemble.nc')

<xarray.Dataset> Size: 219MB
Dimensions:    (time: 1980, lat: 144, lon: 192, bnds: 2)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * lat        (lat) float64 1kB -89.38 -88.12 -86.88 ... 86.88 88.12 89.38
  * lon        (lon) float64 2kB 0.9375 2.812 4.688 6.562 ... 355.3 357.2 359.1
Dimensions without coordinates: bnds
Data variables:
    pr         (time, lat, lon) float32 219MB ...
    time_bnds  (time, bnds) datetime64[ns] 32kB ...
    lat_bnds   (lat, bnds) float64 2kB ...
    lon_bnds   (lon, bnds) float64 3kB ...
    areacella  (lat, lon) float32 111kB ...
    sftlf      (lat, lon) float32 111kB ...
Attributes: (12/45)
    Conventions:            CF-1.7
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  -164359 -182621 -200883 -219145 -237408 -255670 -...
    cmor_version:           3.4.0
    ...                     ...
    table_id:               Amon
    table_info:             Creation Date:(30 April 2019) MD5:12ec21eb48eb992...
    title:                  ACCESS-CM2 output prepared for CMIP6
    variable_id:            pr
    variant_label:          r10i1p1f1 r1i1p1f1 r2i1p1f1 r3i1p1f1 r4i1p1f1 r5i...
    version:                v20191108 v20191125 v20200306 v20210607 v20220819

In [7]:

def sup_variables(dataset, mip, varls):
    '''load supplementary variables as cell measure/ancillary variable to add to cube'''
    supp_var = dataset.copy()
    for sn in varls:
        supp_var.add_supplementary(short_name=sn, mip=mip)
    
    # not handling errors where dataset not found
    firstcb = list(supp_var.from_files())[0].load()

    cellmeas = [(meas, firstcb.cell_measure_dims(meas)) for meas in firstcb.cell_measures()]
    ancilvars = [(anci, firstcb.ancillary_variable_dims(anci)) for anci in firstcb.ancillary_variables()]

    return cellmeas, ancilvars

In [8]:
# tos, Omon 
var = Dataset(
    short_name='tos', mip='Omon', project='CMIP6',
    activity='CMIP',  dataset='ACCESS-CM2',
    ensemble='*',
    institute='*', grid='*',
    exp='historical', timerange='*'
)

## add supplementary
supp_vars = ['areacello', 'sftof'] #Ofx
cellmeas, ancivars = sup_variables(var, 'Ofx', supp_vars)

cube = var.load() # load var after getting supplementary variables
for area in cellmeas:
    cube.add_cell_measure(*area)
for frac in ancivars:
    cube.add_ancillary_variable(*frac)

iris.save(cube, outpath + f'tos_ACCESS-CM2_ensemble.nc')

## Find all available models
- Using esmvalcore dataset searching with wildcards

In [9]:

var = Dataset(
    short_name='ts', mip='Amon', project='CMIP6',
    activity='CMIP',  dataset='*',
    ensemble='*',
    institute='*', grid='*',
    exp='historical', timerange='*'
)


datasets = list(var.from_files())
print(len(datasets))

648


### Format for recipe

In [10]:
from esmvalcore.dataset import datasets_to_recipe
import yaml

for dataset in datasets:
    dataset['diagnostic'] = 'diag' # dummy fill
print(yaml.safe_dump(datasets_to_recipe(datasets)))

datasets:
- dataset: ACCESS-CM2
  ensemble: r(1:10)i1p1f1
  grid: gn
  institute: CSIRO-ARCCSS
  timerange: 185001/201412
- dataset: ACCESS-ESM1-5
  ensemble: r(1:40)i1p1f1
  grid: gn
  institute: CSIRO
  timerange: 185001/201412
- dataset: AWI-CM-1-1-MR
  ensemble: r(1:5)i1p1f1
  grid: gn
  institute: AWI
  timerange: 185001/201412
- dataset: AWI-ESM-1-1-LR
  ensemble: r1i1p1f1
  grid: gn
  institute: AWI
  timerange: 185001/201412
- dataset: BCC-CSM2-MR
  ensemble: r(1:3)i1p1f1
  grid: gn
  institute: BCC
  timerange: 185001/201412
- dataset: BCC-ESM1
  ensemble: r(1:3)i1p1f1
  grid: gn
  institute: BCC
  timerange: 185001/201412
- dataset: CAMS-CSM1-0
  ensemble: r1i1p1f2
  grid: gn
  institute: CAMS
  timerange: 185001/201412
- dataset: CAMS-CSM1-0
  ensemble: r(1:2)i1p1f1
  grid: gn
  institute: CAMS
  timerange: 185001/201412
- dataset: CAS-ESM2-0
  ensemble: r(1:3)i1p1f1
  grid: gn
  institute: CAS
  timerange: 185001/201412
- dataset: CESM2
  ensemble: r(1:11)i1p1f1
  grid: gn


## Recipe using wildcards
Below is an example of an ESMValTool recipe.
This will find all available models and run mean statistics grouping by ensemble statistics.
The output filled yaml file will have all datasets found. However, the recipe run may fail on resolving some data issues such as concatenating incomplete time series and differing coordinates. 

```yaml
documentation:
  title: CMIP6 multi model mean
  authors:
    - chun_felicity

datasets:
  - project: CMIP6
    exp: historical
    dataset: '*'
    institute: '*'
    ensemble: '*'
    grid: '*'
  
preprocessors:
  all_preprocessor:
    regrid:
      target_grid: 1x1
      scheme: linear
    ensemble_statistics:
      statistics: [mean]
    multi_model_statistics:
      span: overlap
      statistics: [mean]
      groupby: [ensemble_statistics]
      keep_input_datasets: false
      ignore_scalar_coords: true

diagnostics:
  diagnostic_metrics:
    description: run preprocessors
    variables:
      ts:
        mip: Amon
        preprocessor: all_preprocessor
      pr:
        mip: Amon
        preprocessor: all_preprocessor

    scripts: null
```